In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [ ]:
train = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
train.head()

In [ ]:
rows, columns = train.shape
rows, columns

In [ ]:
train_info = train.info()
train_info

In [ ]:
train.describe()

categorical_columns = train.select_dtypes(include=['object', 'category']).columns
categorical_summary = train[categorical_columns].describe()
categorical_summary

In [ ]:
list(set(train['expert_consensus'].unique()))